In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from koopmanlib.param_solver import KoopmanParametricDLSolver, KoopmanLinearDLSolver, KoopmanBilinearDLSolver
from koopmanlib.dictionary import PsiNN
from tensorflow.keras.optimizers import Adam
import json
import sys
import numpy as np
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


2024-02-27 21:29:03.798509: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
config_file = 'config_vdpm.json'

In [4]:


# config_file = sys.argv[1]
with open(config_file, 'r') as f:
    config = json.load(f)

data_path = config['data_settings']['data_path']
weights_path = config['nn_settings']['weights_path']

n_psi_train = config['nn_settings']['n_psi_train']
mu_list = config['data_settings']['mu']


target_dim = 2
param_dim = 1

n_psi = 1 + target_dim + n_psi_train

dict_layer_size = config['nn_settings']['dict_layer_size']
K_layer_size_list = config['nn_settings']['K_layer_size']

linear_epochs = config['nn_settings']['linear_epochs']
bilinear_epochs = config['nn_settings']['bilinear_epochs']
pknn_epochs = config['nn_settings']['pknn_epochs']


In [5]:

# class CustomLoss(tf.keras.losses.Loss):
#     def __init__(self, time_step, **kwargs):
#         super().__init__(**kwargs)
#         self.time_step = time_step

#     def call(self, y_true, y_pred):  
   
#         # # mse loss on psi
#         # mse = tf.reduce_mean(tf.square(tf.norm(psi_next-psi_y, axis=-1))) / (self.time_step)**2

#         # mse loss on psi
#         self.mse = tf.square(tf.norm(y_true-y_pred, axis=-1)) / (self.time_step**2)
#         return self.mse
    
# my_loss = CustomLoss(time_step=0.01)


In [6]:

mu = mu_list[0]
K_layer_size = K_layer_size_list[0]

# Load data
dict_data = np.load(os.path.join(
    data_path, 'vdpm_data_mu_'+str(mu)+'.npy'), allow_pickle=True)

data_x = dict_data[()]['data_x']
data_y = dict_data[()]['data_y']
data_u = dict_data[()]['data_u']

# PK-NN
dic_pk = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train, name='psi_nn_layer')
from koopmanlib.K_structure import Model_K_u_Layer, Model_K_u_Layer_One

model_K_u = Model_K_u_Layer_One(layer_sizes=K_layer_size, 
                                n_psi=n_psi,
                                activation='tanh')

solver_pk = KoopmanParametricDLSolver(
    target_dim=target_dim, 
    param_dim=param_dim, 
    n_psi=n_psi, 
    dic=dic_pk, 
    model_K_u=model_K_u)

model_pk, model_K_u_pred_pk = solver_pk.generate_model()


2024-02-27 21:29:06.912856: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-02-27 21:29:06.912908: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: neon
2024-02-27 21:29:06.912914: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: neon
2024-02-27 21:29:06.913032: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 465.19.1
2024-02-27 21:29:06.913053: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 465.19.1
2024-02-27 21:29:06.913058: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 465.19.1
2024-02-27 21:29:06.913420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performa

In [7]:
model_pk.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 tf.slice (TFOpLambda)          (None, 1)            0           ['input_3[0][0]']                
                                                                                                  
 input_4 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 tf.ones_like (TFOpLambda)      (None, 1)            0           ['tf.slice[0][0]']               
                                                                                              

In [8]:

zeros_data_y_train = tf.zeros_like(dic_pk(data_y))


In [13]:

lr = 1e-5

opt = Adam(lr)
model_pk.compile(optimizer=opt, loss='mse')

# lr_callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
#                                                     factor=0.1,
#                                                     patience=200,
#                                                     verbose=0,
#                                                     mode='auto',
#                                                     min_delta=0.0001,
#                                                     cooldown=0,
#                                                     min_lr=1e-6)


In [14]:
log_interval = 20
lr_decay_factor = 0.8
from tqdm.keras import TqdmCallback

losses = []
for i in range(50):
    print('number of the outer loop:', i)

    model_pk.get_layer('K_u_pred').trainable = False
    model_pk.get_layer('DicNN').trainable = True
    
    history_dic = model_pk.fit(x=[data_x, data_y, data_u],
                    y=zeros_data_y_train,
                    epochs=10,
                    batch_size=200,
                    callbacks=TqdmCallback(verbose=1),
                    verbose=0)
    
    model_pk.get_layer('K_u_pred').trainable = True
    model_pk.get_layer('DicNN').trainable = False

    history_K_u = model_pk.fit(x=[data_x, data_y, data_u],
                    y=zeros_data_y_train,
                    epochs=20,
                    batch_size=200,
                    callbacks=TqdmCallback(verbose=1),
                    verbose=0)
    
    if i % log_interval == 0:
        losses.append(history_K_u.history['loss'][-1])

        # Adjust learning rate:
        if len(losses) > 2:
            if losses[-1] > losses[-2]:
                print("Error increased. Decay learning rate")
                curr_lr = lr_decay_factor * model_pk.optimizer.lr
                model_pk.optimizer.lr = curr_lr

number of the outer loop: 0


100%|██████████| 20/20 [00:05<00:00,  3.62epoch/s, loss=2.78e-9]


number of the outer loop: 1


100%|██████████| 20/20 [00:05<00:00,  3.66epoch/s, loss=2.75e-9]


number of the outer loop: 2


100%|██████████| 20/20 [00:05<00:00,  3.72epoch/s, loss=2.71e-9]


number of the outer loop: 3


 15%|█▌        | 3/20 [00:00<00:05,  3.38epoch/s, loss=2.72e-9]

KeyboardInterrupt: 

In [15]:

model_pk.save_weights(os.path.join(
    weights_path, 'model_pk_vdpm_mu_'+str(mu)+'.h5'))


In [32]:

# Linear Model: Dynamics is $Az +Bu$

dic_linear = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)

solver_linear = KoopmanLinearDLSolver(
    dic=dic_linear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi)

model_linear = solver_linear.build_model()

solver_linear.build(model_linear,
                    data_x,
                    data_u,
                    data_y,
                    zeros_data_y_train,
                    epochs=linear_epochs,
                    batch_size=200,
                    lr=0.0001,
                    log_interval=20,
                    lr_decay_factor=0.1)


Epoch 1/10
125/125 [==============================] - 0s 2ms/step - loss: 0.7172 - lr: 1.0000e-04
Epoch 2/10
125/125 [==============================] - 0s 2ms/step - loss: 0.5688 - lr: 1.0000e-04
Epoch 3/10
125/125 [==============================] - 0s 2ms/step - loss: 0.4750 - lr: 1.0000e-04
Epoch 4/10
125/125 [==============================] - 0s 2ms/step - loss: 0.4141 - lr: 1.0000e-04
Epoch 5/10
125/125 [==============================] - 0s 2ms/step - loss: 0.3730 - lr: 1.0000e-04
Epoch 6/10
125/125 [==============================] - 0s 2ms/step - loss: 0.3434 - lr: 1.0000e-04
Epoch 7/10
125/125 [==============================] - 0s 2ms/step - loss: 0.3205 - lr: 1.0000e-04
Epoch 8/10
125/125 [==============================] - 0s 2ms/step - loss: 0.3018 - lr: 1.0000e-04
Epoch 9/10
125/125 [==============================] - 0s 2ms/step - loss: 0.2860 - lr: 1.0000e-04
Epoch 10/10
125/125 [==============================] - 0s 2ms/step - loss: 0.2725 - lr: 1.0000e-04
number of the outer

KeyboardInterrupt: 

In [ ]:

    # model_linear.save_weights(os.path.join(
    #     weights_path, 'model_linear_vdpm_mu_'+str(mu)+'.h5'))



In [23]:

# Bilinear Model: Dynamics is $Az + \sum_{i=1}^{N_{u}}B_{i}zu_{i}$

dic_bilinear = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)

solver_bilinear = KoopmanBilinearDLSolver(
    dic=dic_bilinear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi)

model_bilinear = solver_bilinear.build_model()

solver_bilinear.build(model_bilinear,
                        data_x,
                        data_u,
                        data_y,
                        zeros_data_y_train,
                        epochs=linear_epochs,
                        batch_size=200,
                        lr=0.0001,
                        log_interval=20,
                        lr_decay_factor=0.1)


Epoch 1/10
125/125 [==============================] - 1s 2ms/step - loss: 8.7756e-07 - lr: 1.0000e-04
Epoch 2/10
125/125 [==============================] - 0s 2ms/step - loss: 6.5995e-08 - lr: 1.0000e-04
Epoch 3/10
125/125 [==============================] - 0s 2ms/step - loss: 6.7592e-08 - lr: 1.0000e-04
Epoch 4/10
125/125 [==============================] - 0s 3ms/step - loss: 6.7914e-08 - lr: 1.0000e-04
Epoch 5/10
125/125 [==============================] - 0s 2ms/step - loss: 6.9576e-08 - lr: 1.0000e-04
Epoch 6/10
125/125 [==============================] - 0s 2ms/step - loss: 8.1593e-08 - lr: 1.0000e-04
Epoch 7/10
125/125 [==============================] - 0s 3ms/step - loss: 8.9970e-08 - lr: 1.0000e-04
Epoch 8/10
125/125 [==============================] - 0s 3ms/step - loss: 1.3230e-07 - lr: 1.0000e-04
Epoch 9/10
125/125 [==============================] - 0s 2ms/step - loss: 8.4535e-08 - lr: 1.0000e-04
Epoch 10/10
125/125 [==============================] - 0s 3ms/step - loss: 9.6420e

KeyboardInterrupt: 

In [ ]:

# model_bilinear.save_weights(os.path.join(
#     weights_path, 'model_bilinear_vdpm_mu_'+str(mu)+'.h5'))


In [ ]:

# for mu, K_layer_size in zip(mu_list, K_layer_size_list):
#     load_data_and_train_models(mu, K_layer_size)
#     print('mu = ', mu, 'done')
#     print('K_layer_size = ', K_layer_size, 'done')

# load_data_and_train_models(mu=mu_list[0], K_layer_size=K_layer_size_list[0])
# print('mu = ', mu_list[0], 'done')
# print('K_layer_size = ', K_layer_size_list[0], 'done')

In [ ]:

# def load_data_and_train_models(mu, K_layer_size):

#     # Load data
#     dict_data = np.load(os.path.join(
#         data_path, 'vdpm_data_mu_'+str(mu)+'.npy'), allow_pickle=True)

#     data_x = dict_data[()]['data_x']
#     data_y = dict_data[()]['data_y']
#     data_u = dict_data[()]['data_u']

#     # PK-NN
#     dic_pk = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)
#     from koopmanlib.K_structure import Model_K_u_Layer, Model_K_u_Layer_One

#     model_K_u = Model_K_u_Layer_One(layer_sizes=K_layer_size, 
#                                     n_psi=n_psi,
#                                     activation='tanh')

#     solver_pk = KoopmanParametricDLSolver(
#         target_dim=target_dim, 
#         param_dim=param_dim, 
#         n_psi=n_psi, 
#         dic=dic_pk, 
#         model_K_u=model_K_u)

#     model_pk, model_K_u_pred_pk = solver_pk.generate_model()

#     model_pk.summary()

#     zeros_data_y_train = tf.zeros_like(dic_pk(data_y))

#     model_pk.compile(optimizer=Adam(0.001),
#                     loss='mse')
    
#     # model_pk.compile(optimizer=Adam(0.001),
#     #                 loss=my_loss)

#     lr_callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
#                                                         factor=0.1,
#                                                         patience=200,
#                                                         verbose=0,
#                                                         mode='auto',
#                                                         min_delta=0.0001,
#                                                         cooldown=0,
#                                                         min_lr=1e-6)

#     history = model_pk.fit(x=[data_x, data_y, data_u],
#                         y=zeros_data_y_train,
#                         epochs=pknn_epochs,
#                         batch_size=200,
#                         callbacks=lr_callbacks,
#                         verbose=1)


#     model_pk.save_weights(os.path.join(
#         weights_path, 'model_pk_vdpm_mu_'+str(mu)+'.h5'))

#     # # Linear Model: Dynamics is $Az +Bu$

#     # dic_linear = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)

#     # solver_linear = KoopmanLinearDLSolver(
#     #     dic=dic_linear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi)

#     # model_linear = solver_linear.build_model()

#     # solver_linear.build(model_linear,
#     #                     data_x,
#     #                     data_u,
#     #                     data_y,
#     #                     zeros_data_y_train,
#     #                     epochs=linear_epochs,
#     #                     batch_size=200,
#     #                     lr=0.0001,
#     #                     log_interval=20,
#     #                     lr_decay_factor=0.1)

#     # model_linear.save_weights(os.path.join(
#     #     weights_path, 'model_linear_vdpm_mu_'+str(mu)+'.h5'))


#     # # Bilinear Model: Dynamics is $Az + \sum_{i=1}^{N_{u}}B_{i}zu_{i}$

#     # dic_bilinear = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)

#     # solver_bilinear = KoopmanBilinearDLSolver(
#     #     dic=dic_bilinear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi)

#     # model_bilinear = solver_bilinear.build_model()

#     # solver_bilinear.build(model_bilinear,
#     #                       data_x,
#     #                       data_u,
#     #                       data_y,
#     #                       zeros_data_y_train,
#     #                       epochs=linear_epochs,
#     #                       batch_size=200,
#     #                       lr=0.0001,
#     #                       log_interval=20,
#     #                       lr_decay_factor=0.1)

#     # model_bilinear.save_weights(os.path.join(
#     #     weights_path, 'model_bilinear_vdpm_mu_'+str(mu)+'.h5'))

# # for mu, K_layer_size in zip(mu_list, K_layer_size_list):
# #     load_data_and_train_models(mu, K_layer_size)
# #     print('mu = ', mu, 'done')
# #     print('K_layer_size = ', K_layer_size, 'done')

# load_data_and_train_models(mu=mu_list[0], K_layer_size=K_layer_size_list[0])
# print('mu = ', mu_list[0], 'done')
# print('K_layer_size = ', K_layer_size_list[0], 'done')